Here, the performance of the lookup tables optimized in the "example_F_lookup_training_test.ipynb" on state stabilization for longer time durations can be tested.

In [1]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from helpers import (
    generate_random_discrete_state,
    generate_random_bloch_state,
    test_implementations,
    generate_decay_superoperator,
)
import numpy as np
from feedback_grape.utils.fidelity import fidelity
from feedback_grape.utils.operators import sigmam
from library.utils.qubit_chain_1D import embed
from feedback_grape.utils.solver import mesolve
from tqdm import tqdm
import jax
import jax.numpy as jnp

test_implementations()

In [2]:
# Physical parameters
N_chains = 3 # Number of parallel chains to simulate
gamma = 0.05 # Decay constant
evaluation_time_steps = 50 # Number of time steps for evaluation
batch_size = 256 # Number of random states to evaluate in parallel

generate_random_state = generate_random_discrete_state
# Path to save results
results_path = "./evaluation_results/baseline_fidelities.npz"

In [3]:
decay_superoperator = generate_decay_superoperator(N_chains, gamma)

# Evolve all basis states and compute fidelities
keys = jax.random.split(jax.random.PRNGKey(42), batch_size)
states = jnp.array([generate_random_state(key, N_chains, noise_level = 0.0) for key in keys])
target_states = jnp.array([generate_random_state(key, N_chains, noise_level=0.0) for key in keys])

fidelities_each = np.zeros((len(states), evaluation_time_steps+1))
for i, (state, target_state) in enumerate(zip(states, target_states)):
    fidelities_each[i, 0] = fidelity(
        C_target=target_state,
        U_final=state,
        evo_type="density",
    )

def propagate_single_timestep(rho, rho_target):
    tmp = decay_superoperator(rho)

    fid = fidelity(C_target=rho_target, U_final=tmp, evo_type="density")

    return tmp, fid

propagate_single_timestep_vmap = jax.vmap(jax.jit(propagate_single_timestep))

for i in tqdm(range(evaluation_time_steps)):
    states, fid = propagate_single_timestep_vmap(states, target_states)
    fidelities_each[:, i+1] = fid

    #for j, rho in enumerate(states):
    #    assert np.all(np.isclose(rho, rho.conj().T)), "State is not Hermitian"
    #    assert np.isclose(np.trace(rho).real, 1.0), "State is not normalized"
    #    assert np.all(np.linalg.eigvalsh(rho) >= -1e-10), "State is not positive semidefinite"

fidelities_each = fidelities_each.mean(axis=0) # Average over all basis states

# Reshape back to matrix form
np.savez(results_path, fidelities=fidelities_each)

100%|██████████| 50/50 [00:00<00:00, 159.39it/s]


In [4]:
fidelities_each

array([1.        , 0.92600111, 0.86230968, 0.80748996, 0.76030618,
       0.71969473, 0.68474013, 0.65465443, 0.62875943, 0.60647139,
       0.5872879 , 0.57077651, 0.55656503, 0.5443331 , 0.53380498,
       0.52474334, 0.51694391, 0.51023089, 0.50445293, 0.4994798 ,
       0.49519938, 0.49151519, 0.48834418, 0.48561487, 0.48326573,
       0.4812438 , 0.47950352, 0.47800564, 0.4767164 , 0.47560674,
       0.47465165, 0.4738296 , 0.47312205, 0.47251306, 0.4719889 ,
       0.47153774, 0.47114943, 0.47081521, 0.47052754, 0.47027995,
       0.47006684, 0.46988341, 0.46972554, 0.46958965, 0.4694727 ,
       0.46937203, 0.46928539, 0.46921081, 0.46914662, 0.46909138,
       0.46904383])